### util function

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import csr_matrix
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm

# 设置设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def standard_input(X):
    """
    标准化输入，这里直接返回原数据；如有需要，可在此处添加标准化处理
    """
    return X

def load_data_DEF(random_state=42):
    """
    从 CSV 文件中加载数据，并划分训练、验证、测试集，同时构造节点 mask
    """
    # CSV 文件路径（根据实际情况修改）
    path = '/home/gehongfei/project/TabGNN/dataset/DEF.csv'
    
    df = pd.read_csv(path, sep=',')
    
    target_col = 'label'
    if target_col not in df.columns:
        print(f"Error: '{target_col}' column not found in the dataset.")
        return None, None, None, None, None, None, None, None, None, None, None
    
    y = df[target_col]
    if "ID" in df.columns:
        X = df.drop(columns=["ID", target_col])
    else:
        X = df.drop(columns=[target_col])
    
    # 划分训练、验证和测试集
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, random_state=random_state, stratify=y
    )
    X_valid, X_test, y_valid, y_test = train_test_split(
        X_temp, y_temp, test_size=2/3, random_state=random_state, stratify=y_temp
    )
    
    # 创建节点 mask（假设每一行数据代表图中的一个节点）
    num_nodes = len(df)
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask   = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask  = torch.zeros(num_nodes, dtype=torch.bool)
    
    train_mask[X_train.index] = True
    val_mask[X_valid.index]   = True
    test_mask[X_test.index]   = True
    
    # 如果需要标准化，可在此处实现
    X = standard_input(X)
    X_train = standard_input(X_train)
    X_valid = standard_input(X_valid)
    X_test  = standard_input(X_test)
    
    return X, y, X_train, X_valid, X_test, y_train, y_valid, y_test, train_mask, val_mask, test_mask

def compute_adjacency_matrix_by_prototypes(X_train, X_valid, X_test, y_train, y_valid,
                                             n_clusters=1000, n_estimators=50, max_depth=None,
                                             random_state=42):
    """
    利用原型构造邻接矩阵的两步方法：
      ① 对所有样本聚类（KMeans），得到 n_clusters 个原型中心；
      ② 步骤一：利用训练+验证数据训练的随机森林，对原型集合构造边，得到原型相似性矩阵 A_proto（尺寸 n_clusters×n_clusters）；  
          步骤二：对于每个原型对应的簇，利用该原型和其簇内样本构造局部数据集，利用随机森林构造局部边（仅计算簇内样本间的相似性）；  
          将两部分边合并，得到全局邻接矩阵。
    返回:
      - 全局邻接矩阵（csr_matrix 格式）
      - 每个样本的聚类标签（长度为样本数的数组）
    """
    # ① 合并所有样本，便于后续操作
    X_all = pd.concat([X_train, X_valid, X_test], axis=0)
    num_samples = X_all.shape[0]
    
    # 聚类：显式设置 n_init 以避免 FutureWarning
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=10)
    cluster_labels = kmeans.fit_predict(X_all)
    prototypes = kmeans.cluster_centers_  # shape: (n_clusters, num_features)
    
    # ② 步骤一：利用训练+验证数据构造原型之间的边
    X_train_valid = pd.concat([X_train, X_valid], axis=0)
    y_train_valid = pd.concat([y_train, y_valid], axis=0)
    
    rf_proto = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                      random_state=random_state)
    rf_proto.fit(X_train_valid, y_train_valid)
    
    # 将原型转换为 DataFrame，确保列名与 X_all 一致
    df_prototypes = pd.DataFrame(prototypes, columns=X_all.columns)
    leaf_indices_proto = rf_proto.apply(df_prototypes)  # shape: (n_clusters, n_trees)
    
    A_proto = np.zeros((n_clusters, n_clusters))
    num_trees = leaf_indices_proto.shape[1]
    for tree_idx in tqdm(range(num_trees), desc="计算原型相似性"):
        leaf_to_protos = {}
        for proto_idx, leaf_id in enumerate(leaf_indices_proto[:, tree_idx]):
            leaf_to_protos.setdefault(leaf_id, []).append(proto_idx)
        for proto_list in leaf_to_protos.values():
            # 两两累加得分
            for i in proto_list:
                for j in proto_list:
                    if i != j:
                        A_proto[i, j] += 1
    if A_proto.max() > 0:
        A_proto = A_proto / A_proto.max()  # 归一化至 [0,1]
    
    # ③ 构造全局邻接矩阵，初始化为零矩阵
    global_adj = np.zeros((num_samples, num_samples))
    
    # 对于不同簇的样本，直接赋值原型相似性（步骤一的边）
    # 即：对于任意两个样本 i, j（所属簇分别为 a 和 b，且 a != b），
    # global_adj[i, j] = A_proto[a, b]
    for cluster_a in tqdm(range(n_clusters), desc="赋值不同簇之间的边"):
        idx_a = np.where(cluster_labels == cluster_a)[0]
        for cluster_b in range(cluster_a + 1, n_clusters):
            idx_b = np.where(cluster_labels == cluster_b)[0]
            weight = A_proto[cluster_a, cluster_b]
            if weight > 0:
                global_adj[np.ix_(idx_a, idx_b)] = weight
                global_adj[np.ix_(idx_b, idx_a)] = weight

    # ④ 步骤二：对于每个簇，计算局部相似性边（仅针对同一簇内的样本）
    for cluster in tqdm(range(n_clusters), desc="计算同簇局部边"):
        indices = np.where(cluster_labels == cluster)[0]
        if len(indices) < 2:
            continue  # 如果簇内样本不足2个，则跳过
        
        # 构造局部数据集：第一行为该簇的原型，其余为该簇内样本
        df_proto = pd.DataFrame([prototypes[cluster]], columns=X_all.columns)
        df_samples = X_all.iloc[indices].reset_index(drop=True)
        local_data = pd.concat([df_proto, df_samples], ignore_index=True)
        # 构造标签：原型设为 0，簇内样本设为 1
        local_labels = [0] + [1] * len(indices)
        
        try:
            rf_local = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                              random_state=random_state)
            rf_local.fit(local_data, local_labels)
        except Exception as e:
            print(f"局部随机森林训练失败（簇 {cluster}）：{e}")
            continue
        
        leaf_indices_local = rf_local.apply(local_data)  # shape: (num_points, n_trees_local)
        A_local = np.zeros((len(indices) + 1, len(indices) + 1))
        num_trees_local = leaf_indices_local.shape[1]
        for tree_idx in range(num_trees_local):
            leaf_to_nodes = {}
            for node_idx, leaf_id in enumerate(leaf_indices_local[:, tree_idx]):
                leaf_to_nodes.setdefault(leaf_id, []).append(node_idx)
            for node_list in leaf_to_nodes.values():
                for i_local in node_list:
                    for j_local in node_list:
                        if i_local != j_local:
                            A_local[i_local, j_local] += 1
        if A_local.max() > 0:
            A_local = A_local / A_local.max()
        # 只保留样本之间的相似性（去除原型对应的行和列，即索引 1:）
        A_local_samples = A_local[1:, 1:]
        # 将局部相似性累加到全局邻接矩阵的相应位置（仅针对同一簇内的样本）
        for i, global_i in enumerate(indices):
            for j, global_j in enumerate(indices):
                if global_i != global_j:
                    global_adj[global_i, global_j] += A_local_samples[i, j]
    
    # ⑤ 对全局邻接矩阵归一化（如果最大值大于0）
    if global_adj.max() > 0:
        global_adj = global_adj / global_adj.max()
    
    return csr_matrix(global_adj), cluster_labels

def adjacency_to_edge_index(adj_matrix, cluster_labels, proto_threshold=0.05, local_threshold=0.05):
    """
    将稀疏邻接矩阵转换为边索引，并根据两种不同的阈值进行二值化：
      - 跨簇（原型）边：仅保留边权大于 proto_threshold 的边；
      - 同簇内（局部）边：仅保留边权大于 local_threshold 的边。
    
    参数:
      adj_matrix: scipy.sparse 格式的邻接矩阵
      cluster_labels: 长度为节点数的数组，记录每个节点所属的聚类标签
      proto_threshold: 用于过滤跨簇边（原型边）的阈值
      local_threshold: 用于过滤同簇内边（局部边）的阈值
    
    返回:
      edge_index: torch.tensor 格式的边索引，形状为 [2, num_edges]
    """
    if not isinstance(cluster_labels, np.ndarray):
        cluster_labels = np.array(cluster_labels)
    
    # 将稀疏矩阵转换为 NumPy 数组
    adj_array = adj_matrix.toarray()
    
    # 构造同簇与跨簇的掩码
    same_mask = (cluster_labels[:, None] == cluster_labels[None, :])
    cross_mask = ~same_mask
    
    # 分别对同簇和跨簇位置应用不同的阈值
    binary_adj = np.zeros_like(adj_array, dtype=int)
    binary_adj[same_mask] = (adj_array[same_mask] > local_threshold).astype(int)
    binary_adj[cross_mask] = (adj_array[cross_mask] > proto_threshold).astype(int)
    
    # 转换为 COO 格式后构造 edge_index
    coo = csr_matrix(binary_adj).tocoo()
    edge_index = torch.tensor(np.vstack((coo.row, coo.col)), dtype=torch.long)
    
    print("邻接矩阵转换完成！Edge index 维度:", edge_index.shape)
    return edge_index


X, y, X_train, X_valid, X_test, y_train, y_valid, y_test, train_mask, val_mask, test_mask = load_data_DEF()
print("数据加载完成！")

# 计算基于原型构造的全局邻接矩阵，并获得每个节点的聚类标签
print("开始计算基于原型中心的邻接矩阵...")
adj_matrix, cluster_labels = compute_adjacency_matrix_by_prototypes(
    X_train, X_valid, X_test, y_train, y_valid,
    n_clusters=1000, n_estimators=50, max_depth=None, random_state=42
)
print("邻接矩阵计算完成，形状为：", adj_matrix.shape)

# 将邻接矩阵转换为 edge_index 格式，并分别设置跨簇和同簇边的阈值
edge_index = adjacency_to_edge_index(adj_matrix, cluster_labels, proto_threshold=0.05, local_threshold=0.15)
print("邻接矩阵转换完成！")
print("Edge index 维度:", edge_index.shape)


数据加载完成！
开始计算基于原型中心的邻接矩阵...


计算同簇局部边: 100%|███████████████████████| 1000/1000 [01:47<00:00,  9.33it/s]


邻接矩阵计算完成，形状为： (30000, 30000)
邻接矩阵转换完成！Edge index 维度: torch.Size([2, 102846068])
邻接矩阵转换完成！
Edge index 维度: torch.Size([2, 102846068])


In [21]:
# 将邻接矩阵转换为 edge_index 格式，并分别设置跨簇和同簇边的阈值
edge_index = adjacency_to_edge_index(adj_matrix, cluster_labels, proto_threshold=0.15, local_threshold=0.35)
print("邻接矩阵转换完成！")
print("Edge index 维度:", edge_index.shape)

邻接矩阵转换完成！Edge index 维度: torch.Size([2, 51051092])
邻接矩阵转换完成！
Edge index 维度: torch.Size([2, 51051092])


In [19]:
import numpy as np

# 随机选择 5 个非零值进行打印
sampled_values = np.random.choice(adj_matrix.data, size=min(5, len(adj_matrix.data)), replace=False)
print("随机抽取的5个非零邻接矩阵值：", sampled_values)


随机抽取的5个非零邻接矩阵值： [0.075 0.075 0.025 0.025 0.025]


### Batch-Based Optimization

In [6]:
import itertools
import random
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.utils import k_hop_subgraph
from tqdm import tqdm  # 用于显示训练进度

# 假设设备定义如下
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#########################################
# 1. 定义模型
#########################################
# GraphSAGE 模型（包含残差结构和 dropout，每层隐藏单元数递减至上一层的3/4）
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout_rate=0.5):
        """
        参数说明：
          in_channels: 输入特征维度
          hidden_channels: 第一层的隐藏单元数
          out_channels: 输出类别数
          num_layers: 图卷积层的总层数（至少为 1）
          dropout_rate: dropout 概率
        """
        super(GraphSAGE, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.residuals = torch.nn.ModuleList()
        # 第一层：从 in_channels 到 hidden_channels
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        if in_channels != hidden_channels:
            self.residuals.append(torch.nn.Linear(in_channels, hidden_channels))
        else:
            self.residuals.append(torch.nn.Identity())
        current_hidden = hidden_channels
        # 后续每一层的隐藏单元数为上一层的 3/4（向下取整，最小为 1）
        for _ in range(num_layers - 1):
            next_hidden = max(1, int(current_hidden * 3 / 4))
            self.convs.append(SAGEConv(current_hidden, next_hidden))
            if current_hidden != next_hidden:
                self.residuals.append(torch.nn.Linear(current_hidden, next_hidden))
            else:
                self.residuals.append(torch.nn.Identity())
            current_hidden = next_hidden
        # 全连接层：将最后一层的隐藏向量映射到输出类别
        self.fc = torch.nn.Linear(current_hidden, out_channels)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def encode(self, x, edge_index):
        """提取节点表示：依次通过图卷积层、残差连接、ReLU 和 dropout（适用于全图或子图）"""
        for conv, res in zip(self.convs, self.residuals):
            out = conv(x, edge_index)
            res_x = res(x)
            x = self.dropout(torch.relu(out + res_x))
        return x

    def forward(self, data):
        x = self.encode(data.x, data.edge_index)
        x = self.fc(x)
        return x

# GAT 模型（包含残差结构和 dropout，每层隐藏单元数递减至上一层的3/4）
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout_rate=0.5, heads=1, concat=True):
        """
        参数说明：
          in_channels: 输入特征维度
          hidden_channels: 第一层的隐藏单元数
          out_channels: 输出类别数
          num_layers: 图卷积层的总层数（至少为 1）
          dropout_rate: dropout 概率（同时用于 GATConv 内部 dropout）
          heads: 注意力头的数量
          concat: 是否拼接多头输出（True）或取平均（False）
        """
        super(GAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.residuals = torch.nn.ModuleList()
        # 第一层
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout_rate, concat=concat))
        out_dim = hidden_channels * heads if concat else hidden_channels
        if in_channels != out_dim:
            self.residuals.append(torch.nn.Linear(in_channels, out_dim))
        else:
            self.residuals.append(torch.nn.Identity())
        current_dim = out_dim
        # 后续层
        for _ in range(num_layers - 1):
            next_hidden = max(1, int(current_dim * 3 / 4))
            self.convs.append(GATConv(current_dim, next_hidden, heads=heads, dropout=dropout_rate, concat=concat))
            new_out_dim = next_hidden * heads if concat else next_hidden
            if current_dim != new_out_dim:
                self.residuals.append(torch.nn.Linear(current_dim, new_out_dim))
            else:
                self.residuals.append(torch.nn.Identity())
            current_dim = new_out_dim
        self.fc = torch.nn.Linear(current_dim, out_channels)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def encode(self, x, edge_index):
        """提取节点表示：依次通过 GAT 层、残差连接、ReLU 和 dropout"""
        for conv, res in zip(self.convs, self.residuals):
            out = conv(x, edge_index)
            res_x = res(x)
            x = self.dropout(torch.relu(out + res_x))
        return x

    def forward(self, data):
        x = self.encode(data.x, data.edge_index)
        x = self.fc(x)
        return x

#########################################
# 2. 定义损失函数
#########################################
# Focal Loss（用于处理类别不平衡）
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  
        self.reduction = reduction
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            if isinstance(self.alpha, (list, np.ndarray)):
                alpha = inputs.new_tensor(self.alpha)
            else:
                alpha = self.alpha
            at = alpha.gather(0, targets.data)
            ce_loss = at * ce_loss
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean() if self.reduction == "mean" else focal_loss.sum()

# 修改后的普通对比学习损失（不使用 mask）
class SupConLoss(torch.nn.Module):
    def __init__(self, temperature=0.07):
        """
        Args:
            temperature: 温度参数
        """
        super(SupConLoss, self).__init__()
        self.temperature = temperature

    def forward(self, features):
        """
        Args:
            features: [batch_size, n_views, feature_dim]
                      要求每个样本至少有两个视图，视图之间互为正样本，其余样本均为负样本。
        Returns:
            对比损失（InfoNCE Loss）
        """
        device = features.device
        if len(features.shape) < 3:
            raise ValueError('`features` 需要形状为 [batch_size, n_views, feature_dim]')
        batch_size, n_views, feature_dim = features.shape

        # 将多个视图拼接为 [batch_size*n_views, feature_dim]
        features = features.view(batch_size * n_views, feature_dim)
        # 对每个特征进行 L2 归一化
        features = torch.nn.functional.normalize(features, p=2, dim=1)

        # 计算相似度矩阵，形状 [batch_size*n_views, batch_size*n_views]
        similarity_matrix = torch.matmul(features, features.T) / self.temperature

        # 构造正样本掩码：同一原始样本（即同一 batch 中的不同视图）的两两之间为正样本
        labels = torch.arange(batch_size, device=device).repeat_interleave(n_views)
        mask = torch.eq(labels.unsqueeze(1), labels.unsqueeze(0)).float()
        # 去除自身对比（对角线置 0）
        self_mask = torch.eye(mask.shape[0], device=device)
        mask = mask - self_mask

        # 计算 exp(similarity)
        exp_sim = torch.exp(similarity_matrix) * (1 - self_mask)
        # 对每个 anchor，分母为除自身外所有样本的 exp(sim)
        denom = exp_sim.sum(dim=1, keepdim=True) + 1e-8

        # 计算仅正样本对的对数概率
        log_prob = similarity_matrix - torch.log(denom)
        numerator = (mask * log_prob).sum(dim=1)
        # 正样本个数（防止除 0）
        pos_count = mask.sum(dim=1) + 1e-8
        loss = - (numerator / pos_count)
        loss = loss.mean()
        return loss

#########################################
# 3. 数据增强方法
#########################################
# 原始的特征扰动（用于 "feature" 增强方式）
def perturb_features(features, noise_level=0.1):
    """对特征进行扰动，生成增强视图"""
    noise = torch.randn_like(features) * noise_level
    return features + noise

# 节点丢弃
def augment_node_drop(features, edge_index, drop_prob=0.1):
    """
    节点丢弃：以一定概率丢弃节点（将被丢弃节点的特征置零，
    同时删除其相关边，但保持节点的序号不变）。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_nodes = features.shape[0]
    keep_mask = (torch.rand(num_nodes, device=features.device) > drop_prob)
    features_aug = features * keep_mask.unsqueeze(1).float()
    src, dst = edge_index
    valid_edge_mask = keep_mask[src] & keep_mask[dst]
    edge_index_aug = edge_index[:, valid_edge_mask]
    return features_aug, edge_index_aug

# 边丢弃
def augment_edge_drop(features, edge_index, drop_prob=0.1):
    """
    边丢弃：以一定概率删除边，但保留所有节点和原始特征。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_edges = edge_index.shape[1]
    mask = (torch.rand(num_edges, device=edge_index.device) > drop_prob)
    edge_index_aug = edge_index[:, mask]
    return features, edge_index_aug

# 边扰动
def augment_edge_perturb(features, edge_index, drop_prob=0.1):
    """
    边扰动：先以一定概率删除部分边，再随机添加一些新的边，
    添加的新边数量与被删除边的数量相当。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_edges = edge_index.shape[1]
    num_nodes = features.shape[0]
    mask = (torch.rand(num_edges, device=edge_index.device) > drop_prob)
    edge_index_dropped = edge_index[:, mask]
    num_dropped = num_edges - mask.sum().item()
    if num_dropped > 0:
        new_edges = torch.randint(0, num_nodes, (2, num_dropped), device=features.device)
        edge_index_aug = torch.cat([edge_index_dropped, new_edges], dim=1)
    else:
        edge_index_aug = edge_index_dropped
    return features, edge_index_aug

def augment_data(data, aug_method="feature", aug_ratio=0.1):
    """
    根据指定的增强方式对图数据进行增强，返回增强后的节点特征和 edge_index。
    参数:
      aug_method: "feature"（特征扰动）, "node_drop", "edge_drop", "edge_perturb"
      aug_ratio: 控制增强强度（例如噪声水平或丢弃比例）
    """
    if aug_method == "feature":
        x_aug = perturb_features(data.x, noise_level=aug_ratio)
        edge_index_aug = data.edge_index  # 图结构不变
        return x_aug, edge_index_aug
    elif aug_method == "node_drop":
        return augment_node_drop(data.x, data.edge_index, drop_prob=aug_ratio)
    elif aug_method == "edge_drop":
        return augment_edge_drop(data.x, data.edge_index, drop_prob=aug_ratio)
    elif aug_method == "edge_perturb":
        return augment_edge_perturb(data.x, data.edge_index, drop_prob=aug_ratio)
    else:
        raise ValueError(f"Unknown augmentation method: {aug_method}")

#########################################
# 3.1 辅助函数：提取 mini-batch 子图
#########################################
def get_mini_batch_data(data, batch_node_idx, num_hops):
    """
    对单图中一小批节点（batch_node_idx）提取 k-hop 子图。
    使用 torch_geometric.utils.k_hop_subgraph 进行子图提取，并 relabel 节点。
    返回：
      sub_data: 包含子图的 Data 对象（x, edge_index, y 以及 mask 可选）
      mapping: 一个长整型张量，指示子图中哪一部分对应原始 batch_node_idx（目标节点在子图中的索引）
    """
    # k_hop_subgraph 返回：(subset, sub_edge_index, mapping, edge_mask)
    subset, sub_edge_index, mapping, _ = k_hop_subgraph(
        node_idx=batch_node_idx, num_hops=num_hops, edge_index=data.edge_index, relabel_nodes=True)
    sub_data = Data(x=data.x[subset], edge_index=sub_edge_index, y=data.y[subset])
    # 如果原始 data 定义了 train/val/test mask，则在子图中也保留（注意：mask 按 subset 索引提取）
    if hasattr(data, 'train_mask'):
        sub_data.train_mask = data.train_mask[subset]
    if hasattr(data, 'val_mask'):
        sub_data.val_mask = data.val_mask[subset]
    if hasattr(data, 'test_mask'):
        sub_data.test_mask = data.test_mask[subset]
    return sub_data, mapping

#########################################
# 4. 训练函数（预训练 + 微调）——mini-batch 版（基于 k_hop_subgraph）
#########################################
def pretrain_model(data, model, optimizer, criterion_contrast, num_epochs=200, aug_method="feature", aug_ratio=0.1, batch_size=64):
    """
    预训练阶段：仅使用对比损失训练模型（不计算 Focal Loss）。
    采用 mini-batch 方式，先对训练集中的一批目标节点提取 k-hop 子图，
    再对该子图进行数据增强生成两视图，最后计算对比损失（仅对 batch 中的目标节点计算）。
    使用 tqdm 显示每个 epoch 以及 batch 的进度和损失。
    """
    best_f1 = 0.0
    best_model_state = None
    train_idx = data.train_mask.nonzero(as_tuple=False).view(-1).tolist()
    num_hops = len(model.convs)  # 以模型层数作为子图的 hop 数

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        count = 0
        loader = DataLoader(train_idx, batch_size=batch_size, shuffle=True)
        # 使用 tqdm 包裹 batch 迭代
        pbar = tqdm(loader, desc=f"Pretrain Epoch {epoch+1}/{num_epochs}", leave=False)
        for batch in pbar:
            batch = batch.clone().detach().to(device)
            sub_data, mapping = get_mini_batch_data(data, batch, num_hops)
            sub_data = sub_data.to(device)
            optimizer.zero_grad()
            x_aug1, edge_index1 = augment_data(sub_data, aug_method, aug_ratio)
            x_aug2, edge_index2 = augment_data(sub_data, aug_method, aug_ratio)
            embedding_aug1 = model.encode(x_aug1, edge_index1)
            embedding_aug2 = model.encode(x_aug2, edge_index2)
            target_emb1 = embedding_aug1[mapping]
            target_emb2 = embedding_aug2[mapping]
            features_aug = torch.stack([target_emb1, target_emb2], dim=1)
            loss = criterion_contrast(features_aug)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * len(batch)
            count += len(batch)
            avg_loss = total_loss / count
            pbar.set_postfix(batch_loss=f"{loss.item():.4f}", avg_loss=f"{avg_loss:.4f}")
        print(f"Pretrain Epoch {epoch+1}/{num_epochs}, Contrast Loss: {avg_loss:.4f}")

        # 验证阶段
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out[data.val_mask].argmax(dim=1)
            true = data.y[data.val_mask]
            val_f1 = f1_score(true.cpu(), preds.cpu(), average="macro")
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    return best_model_state

def fine_tune_model(data, model, optimizer, criterion_focal, num_epochs=50, batch_size=64):
    """
    微调阶段：仅使用 Focal Loss 进行训练（不计算对比损失）。
    同样采用 mini-batch（基于 k_hop 子图）方式训练，损失仅在目标节点上计算。
    使用 tqdm 实时显示每个 epoch 与 batch 的训练进度和损失信息。
    """
    best_f1 = 0.0
    best_model_state = None
    train_idx = data.train_mask.nonzero(as_tuple=False).view(-1).tolist()
    num_hops = len(model.convs)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        count = 0
        loader = DataLoader(train_idx, batch_size=batch_size, shuffle=True)
        pbar = tqdm(loader, desc=f"Fine-tune Epoch {epoch+1}/{num_epochs}", leave=False)
        for batch in pbar:
            batch = batch.clone().detach().to(device)
            sub_data, mapping = get_mini_batch_data(data, batch, num_hops)
            sub_data = sub_data.to(device)
            optimizer.zero_grad()
            out = model(sub_data)
            loss = criterion_focal(out[mapping], sub_data.y[mapping])
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * len(batch)
            count += len(batch)
            avg_loss = total_loss / count
            pbar.set_postfix(batch_loss=f"{loss.item():.4f}", avg_loss=f"{avg_loss:.4f}")
        print(f"Fine-tune Epoch {epoch+1}/{num_epochs}, Focal Loss: {avg_loss:.4f}")

        # 验证阶段
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out[data.val_mask].argmax(dim=1)
            true = data.y[data.val_mask]
            val_f1 = f1_score(true.cpu(), preds.cpu(), average="macro")
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    return best_model_state

def two_stage_train_model(data, model, optimizer, optimizer_ft, criterion_focal, criterion_contrast,
                          pretrain_epochs, finetune_epochs, aug_method="feature", aug_ratio=0.1,
                          batch_size=64):
    """
    两阶段训练：
      第一阶段：预训练（仅用对比损失，mini-batch 方式）；
      第二阶段：微调（仅用分类损失，mini-batch 方式）。
    """
    print("========== 开始预训练阶段 ==========")
    best_pretrain_state = pretrain_model(data, model, optimizer, criterion_contrast,
                                         num_epochs=pretrain_epochs,
                                         aug_method=aug_method, aug_ratio=aug_ratio,
                                         batch_size=batch_size)
    model.load_state_dict(best_pretrain_state)

    print("========== 开始微调阶段 ==========")
    best_finetune_state = fine_tune_model(data, model, optimizer_ft,
                                          criterion_focal, num_epochs=finetune_epochs,
                                          batch_size=batch_size)
    return best_finetune_state

#########################################
# 5. 封装随机采样超参数组合的函数
#########################################
def get_continuous_candidates(start, stop, step, decimals):
    """
    生成从 start 到 stop（含）之间，以 step 为步长的候选列表，并保留指定小数位数。
    """
    num_steps = int((stop - start) / step) + 1
    return [round(start + i * step, decimals) for i in range(num_steps)]

def get_random_hyperparameter_combinations(n_iter):
    """
    随机生成 n_iter 个超参数组合，每个组合包含：
      (threshold, random_state, num_layers, hidden_channels, finetune_lr, pretrain_lr,
       gamma, alpha_value, aug_method, aug_ratio, pretrain_epochs, temperature,
       model_type, dropout_rate)
    """
    # 离散变量候选列表
    discrete_candidates = {
        'random_state': list(range(0, 40, 10)),
        'num_layers': [2, 3, 4],
        'hidden_channels': list(range(20, 300, 20)),
        'finetune_lr': [0.001],
        'pretrain_lr': [0.0001, 0.00001, 0.000001],
        'aug_method': ["feature", "node_drop", "edge_drop", "edge_perturb"],
        'pretrain_epochs': [5],
        'model_type': ["GraphSAGE", "GAT"],
        'dropout_rate': [0.1, 0.2, 0.3]
    }

    # 连续变量候选区间及步长（注意：部分变量保留 2 位小数，其余保留 1 位小数）
    continuous_candidates = {
        'threshold': get_continuous_candidates(0.05, 0.2, 0.01, 2),
        'gamma': get_continuous_candidates(1, 4, 0.1, 1),
        # 'alpha_value': get_continuous_candidates(0.1, 0.9, 0.1, 1),
        'alpha_value': [0.22, 0.78],
        'aug_ratio': get_continuous_candidates(0.05, 0.25, 0.01, 2),
        'temperature': get_continuous_candidates(0.05, 0.1, 0.01, 2)
    }

    combinations = []
    for _ in range(n_iter):
        # 随机采样连续变量
        threshold   = random.choice(continuous_candidates['threshold'])
        gamma       = random.choice(continuous_candidates['gamma'])
        alpha_value = random.choice(continuous_candidates['alpha_value'])
        aug_ratio   = random.choice(continuous_candidates['aug_ratio'])
        temperature = random.choice(continuous_candidates['temperature'])
        
        # 随机采样离散变量
        random_state    = random.choice(discrete_candidates['random_state'])
        num_layers      = random.choice(discrete_candidates['num_layers'])
        hidden_channels = random.choice(discrete_candidates['hidden_channels'])
        finetune_lr     = random.choice(discrete_candidates['finetune_lr'])
        pretrain_lr     = random.choice(discrete_candidates['pretrain_lr'])
        aug_method      = random.choice(discrete_candidates['aug_method'])
        pretrain_epochs = random.choice(discrete_candidates['pretrain_epochs'])
        model_type      = random.choice(discrete_candidates['model_type'])
        dropout_rate    = random.choice(discrete_candidates['dropout_rate'])
        
        # 构造超参数组合（顺序与注释中保持一致）
        combination = (
            threshold,      # 阈值
            random_state,   # 随机种子
            num_layers,     # 图卷积层数
            hidden_channels,# 第一层隐藏单元数
            finetune_lr,    # 微调学习率
            pretrain_lr,    # 预训练学习率
            gamma,          # gamma 参数
            alpha_value,    # alpha 参数
            aug_method,     # 增强方式
            aug_ratio,      # 增强比例
            pretrain_epochs,# 预训练轮数
            temperature,    # 对比学习温度
            model_type,     # 模型类型
            dropout_rate    # dropout 概率
        )
        combinations.append(combination)
        
    return combinations

#########################################
# 6. 随机搜索超参数并评估模型
#########################################
def grid_search(X, y, train_mask, valid_mask, test_mask, n_iter):
    best_acc = 0.0
    best_overall_model_state = None
    best_overall_params = None

    print("Start random search with {} combinations...".format(n_iter))
    hyperparam_combos = get_random_hyperparameter_combinations(n_iter)
    for i, (threshold, random_state, num_layers, hidden_channels, finetune_lr,
            pretrain_lr, gamma, alpha_value, aug_method, aug_ratio, pretrain_epochs, temperature,
            model_type, dropout_rate) in enumerate(hyperparam_combos):
        print(f"\nTesting combination {i+1}: threshold={threshold:.4f}, random_state={random_state}, "
              f"layers={num_layers}, hidden_channels={hidden_channels}, finetune_lr={finetune_lr}, "
              f"pretrain_lr={pretrain_lr}, gamma={gamma:.4f}, alpha={alpha_value:.4f}, aug_method={aug_method}, "
              f"aug_ratio={aug_ratio:.4f}, pretrain_epochs={pretrain_epochs}, temperature={temperature:.4f}, "
              f"model_type={model_type}, dropout_rate={dropout_rate:.4f}")

        # 假设函数 adjacency_to_edge_index 已实现
        edge_index = adjacency_to_edge_index(adjacency_matrix, threshold).to(device)
        X_tensor = torch.tensor(X.values, dtype=torch.float)
        y_tensor = torch.tensor(y.values, dtype=torch.long)
        data = Data(x=X_tensor, y=y_tensor, edge_index=edge_index,
                    train_mask=train_mask, val_mask=valid_mask, test_mask=test_mask).to(device)

        # 根据 model_type 选择模型
        if model_type == "GraphSAGE":
            model = GraphSAGE(in_channels=X.shape[1], hidden_channels=hidden_channels,
                              out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
        elif model_type == "GAT":
            model = GAT(in_channels=X.shape[1], hidden_channels=hidden_channels,
                        out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
        else:
            raise ValueError(f"Unknown model type: {model_type}")
        
        optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_lr, weight_decay=5e-4)
        optimizer_ft = torch.optim.Adam(model.parameters(), lr=finetune_lr, weight_decay=5e-4)

        alpha_list = [1 - alpha_value, alpha_value]
        alpha_tensor = torch.tensor(alpha_list, dtype=torch.float).to(device)
        criterion_focal = FocalLoss(gamma=gamma, alpha=alpha_tensor, reduction="mean")
        criterion_contrast = SupConLoss(temperature=temperature)

        best_model_epoch = two_stage_train_model(data, model, optimizer, optimizer_ft, criterion_focal,
                                                 criterion_contrast, pretrain_epochs=pretrain_epochs, finetune_epochs=10,
                                                 aug_method=aug_method, aug_ratio=aug_ratio, batch_size=4)
        model.load_state_dict(best_model_epoch)
        model.eval()
        with torch.no_grad():
            test_out = model(data)
            preds = test_out[data.test_mask].argmax(dim=1)
            test_acc = accuracy_score(data.y[data.test_mask].cpu(), preds.cpu())
        print(f"Test Accuracy for current combination: {test_acc:.4f}")
        if test_acc > best_acc:
            best_acc = test_acc
            best_overall_model_state = best_model_epoch
            best_overall_params = (threshold, random_state, num_layers, hidden_channels,
                                   finetune_lr, pretrain_lr, gamma, alpha_value, aug_method, aug_ratio,
                                   pretrain_epochs, temperature, model_type, dropout_rate)

    return best_overall_params, best_overall_model_state

#########################################
# 7. 主程序：加载数据、随机搜索超参数、加载最佳模型并评估
#########################################
# 注意：此处假定 X, y, adjacency_matrix, train_mask, valid_mask, test_mask 已经提前加载好，
# 且 edge_index 为形状 [2, num_edges] 的 torch.tensor 对象。

best_params, best_model_state = grid_search(X, y, train_mask, valid_mask, test_mask, n_iter=1)
print("\nBest Hyperparameters:", best_params)

# 解包最佳超参数
(threshold, random_state, num_layers, hidden_channels,
 finetune_lr, pretrain_lr, gamma, alpha_value, aug_method, aug_ratio,
 pretrain_epochs, temperature, model_type, dropout_rate) = best_params

X_tensor = torch.tensor(X.values, dtype=torch.float).to(device)
y_tensor = torch.tensor(y.values, dtype=torch.long).to(device)
edge_index = adjacency_to_edge_index(adjacency_matrix, threshold).to(device)
data = Data(x=X_tensor, y=y_tensor, edge_index=edge_index, train_mask=train_mask.to(device), val_mask=valid_mask.to(device), test_mask=test_mask.to(device))

# 根据最终选出的模型类型构造模型
if model_type == "GraphSAGE":
    model = GraphSAGE(in_channels=X.shape[1], hidden_channels=hidden_channels,
                      out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
elif model_type == "GAT":
    model = GAT(in_channels=X.shape[1], hidden_channels=hidden_channels,
                out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
else:
    raise ValueError(f"Unknown model type: {model_type}")

model.load_state_dict(best_model_state)
model.eval()

with torch.no_grad():
    test_out = model(data)
    preds = test_out[data.test_mask].argmax(dim=1)
    true_labels = data.y[data.test_mask]
    report = classification_report(true_labels.cpu(), preds.cpu(), target_names=["Class 0", "Class 1"], digits=4)
    test_precision = precision_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_recall = recall_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_f1 = f1_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_acc = accuracy_score(true_labels.cpu(), preds.cpu())
    
    print("\nBest Model Classification Report on Test Set:")
    print(report)
    print("Best Model Test Set Metrics:")
    print(f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}, Accuracy: {test_acc:.4f}")


Start random search with 1 combinations...

Testing combination 1: threshold=0.1400, random_state=20, layers=4, hidden_channels=60, finetune_lr=0.001, pretrain_lr=1e-06, gamma=3.1000, alpha=0.2200, aug_method=edge_perturb, aug_ratio=0.2500, pretrain_epochs=5, temperature=0.1000, model_type=GAT, dropout_rate=0.2000
邻接矩阵转换完成！Edge index 维度: torch.Size([2, 60292392])
========== 开始预训练阶段 ==========


RuntimeError: CUDA out of memory. Tried to allocate 13.42 GiB (GPU 0; 23.70 GiB total capacity; 11.96 GiB already allocated; 7.02 GiB free; 14.99 GiB reserved in total by PyTorch)

In [ ]:
### 可能的采样优化 交叉shang损失

In [2]:
import itertools
import random
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, classification_report, accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, GATConv
from torch_geometric.utils import k_hop_subgraph
from tqdm import tqdm  # 用于显示训练进度

# 假设设备定义如下
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#########################################
# 1. 定义模型
#########################################
# GraphSAGE 模型（包含残差结构和 dropout，每层隐藏单元数递减至上一层的3/4）
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout_rate=0.5):
        """
        参数说明：
          in_channels: 输入特征维度
          hidden_channels: 第一层的隐藏单元数
          out_channels: 输出类别数
          num_layers: 图卷积层的总层数（至少为 1）
          dropout_rate: dropout 概率
        """
        super(GraphSAGE, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.residuals = torch.nn.ModuleList()
        # 第一层：从 in_channels 到 hidden_channels
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        if in_channels != hidden_channels:
            self.residuals.append(torch.nn.Linear(in_channels, hidden_channels))
        else:
            self.residuals.append(torch.nn.Identity())
        current_hidden = hidden_channels
        # 后续每一层的隐藏单元数为上一层的 3/4（向下取整，最小为 1）
        for _ in range(num_layers - 1):
            next_hidden = max(1, int(current_hidden * 3 / 4))
            self.convs.append(SAGEConv(current_hidden, next_hidden))
            if current_hidden != next_hidden:
                self.residuals.append(torch.nn.Linear(current_hidden, next_hidden))
            else:
                self.residuals.append(torch.nn.Identity())
            current_hidden = next_hidden
        # 全连接层：将最后一层的隐藏向量映射到输出类别
        self.fc = torch.nn.Linear(current_hidden, out_channels)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def encode(self, x, edge_index):
        """提取节点表示：依次通过图卷积层、残差连接、ReLU 和 dropout（适用于全图或子图）"""
        for conv, res in zip(self.convs, self.residuals):
            out = conv(x, edge_index)
            res_x = res(x)
            x = self.dropout(torch.relu(out + res_x))
        return x

    def forward(self, data):
        x = self.encode(data.x, data.edge_index)
        x = self.fc(x)
        return x

# GAT 模型（包含残差结构和 dropout，每层隐藏单元数递减至上一层的3/4）
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2, dropout_rate=0.5, heads=1, concat=True):
        """
        参数说明：
          in_channels: 输入特征维度
          hidden_channels: 第一层的隐藏单元数
          out_channels: 输出类别数
          num_layers: 图卷积层的总层数（至少为 1）
          dropout_rate: dropout 概率（同时用于 GATConv 内部 dropout）
          heads: 注意力头的数量
          concat: 是否拼接多头输出（True）或取平均（False）
        """
        super(GAT, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.residuals = torch.nn.ModuleList()
        # 第一层
        self.convs.append(GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout_rate, concat=concat))
        out_dim = hidden_channels * heads if concat else hidden_channels
        if in_channels != out_dim:
            self.residuals.append(torch.nn.Linear(in_channels, out_dim))
        else:
            self.residuals.append(torch.nn.Identity())
        current_dim = out_dim
        # 后续层
        for _ in range(num_layers - 1):
            next_hidden = max(1, int(current_dim * 3 / 4))
            self.convs.append(GATConv(current_dim, next_hidden, heads=heads, dropout=dropout_rate, concat=concat))
            new_out_dim = next_hidden * heads if concat else next_hidden
            if current_dim != new_out_dim:
                self.residuals.append(torch.nn.Linear(current_dim, new_out_dim))
            else:
                self.residuals.append(torch.nn.Identity())
            current_dim = new_out_dim
        self.fc = torch.nn.Linear(current_dim, out_channels)
        self.dropout = torch.nn.Dropout(dropout_rate)

    def encode(self, x, edge_index):
        """提取节点表示：依次通过 GAT 层、残差连接、ReLU 和 dropout"""
        for conv, res in zip(self.convs, self.residuals):
            out = conv(x, edge_index)
            res_x = res(x)
            x = self.dropout(torch.relu(out + res_x))
        return x

    def forward(self, data):
        x = self.encode(data.x, data.edge_index)
        x = self.fc(x)
        return x

#########################################
# 2. 定义损失函数
#########################################
# 下面的 FocalLoss 类保留不动，但在微调阶段我们将换用普通的交叉熵损失，
# 因此后续训练中不会使用该类。
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2, alpha=None, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  
        self.reduction = reduction
        self.ce = torch.nn.CrossEntropyLoss(reduction="none")

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        if self.alpha is not None:
            if isinstance(self.alpha, (list, np.ndarray)):
                alpha = inputs.new_tensor(self.alpha)
            else:
                alpha = self.alpha
            at = alpha.gather(0, targets.data)
            ce_loss = at * ce_loss
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean() if self.reduction == "mean" else focal_loss.sum()

# 修改后的普通对比学习损失（不使用 mask）
class SupConLoss(torch.nn.Module):
    def __init__(self, temperature=0.07):
        """
        Args:
            temperature: 温度参数
        """
        super(SupConLoss, self).__init__()
        self.temperature = temperature

    def forward(self, features):
        """
        Args:
            features: [batch_size, n_views, feature_dim]
                      要求每个样本至少有两个视图，视图之间互为正样本，其余样本均为负样本。
        Returns:
            对比损失（InfoNCE Loss）
        """
        device = features.device
        if len(features.shape) < 3:
            raise ValueError('`features` 需要形状为 [batch_size, n_views, feature_dim]')
        batch_size, n_views, feature_dim = features.shape

        # 将多个视图拼接为 [batch_size*n_views, feature_dim]
        features = features.view(batch_size * n_views, feature_dim)
        # 对每个特征进行 L2 归一化
        features = torch.nn.functional.normalize(features, p=2, dim=1)

        # 计算相似度矩阵，形状 [batch_size*n_views, batch_size*n_views]
        similarity_matrix = torch.matmul(features, features.T) / self.temperature

        # 构造正样本掩码：同一原始样本（即同一 batch 中的不同视图）的两两之间为正样本
        labels = torch.arange(batch_size, device=device).repeat_interleave(n_views)
        mask = torch.eq(labels.unsqueeze(1), labels.unsqueeze(0)).float()
        # 去除自身对比（对角线置 0）
        self_mask = torch.eye(mask.shape[0], device=device)
        mask = mask - self_mask

        # 计算 exp(similarity)
        exp_sim = torch.exp(similarity_matrix) * (1 - self_mask)
        # 对每个 anchor，分母为除自身外所有样本的 exp(sim)
        denom = exp_sim.sum(dim=1, keepdim=True) + 1e-8

        # 计算仅正样本对的对数概率
        log_prob = similarity_matrix - torch.log(denom)
        numerator = (mask * log_prob).sum(dim=1)
        # 正样本个数（防止除 0）
        pos_count = mask.sum(dim=1) + 1e-8
        loss = - (numerator / pos_count)
        loss = loss.mean()
        return loss

#########################################
# 3. 数据增强方法
#########################################
# 原始的特征扰动（用于 "feature" 增强方式）
def perturb_features(features, noise_level=0.1):
    """对特征进行扰动，生成增强视图"""
    noise = torch.randn_like(features) * noise_level
    return features + noise

# 节点丢弃
def augment_node_drop(features, edge_index, drop_prob=0.1):
    """
    节点丢弃：以一定概率丢弃节点（将被丢弃节点的特征置零，
    同时删除其相关边，但保持节点的序号不变）。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_nodes = features.shape[0]
    keep_mask = (torch.rand(num_nodes, device=features.device) > drop_prob)
    features_aug = features * keep_mask.unsqueeze(1).float()
    src, dst = edge_index
    valid_edge_mask = keep_mask[src] & keep_mask[dst]
    edge_index_aug = edge_index[:, valid_edge_mask]
    return features_aug, edge_index_aug

# 边丢弃
def augment_edge_drop(features, edge_index, drop_prob=0.1):
    """
    边丢弃：以一定概率删除边，但保留所有节点和原始特征。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_edges = edge_index.shape[1]
    mask = (torch.rand(num_edges, device=edge_index.device) > drop_prob)
    edge_index_aug = edge_index[:, mask]
    return features, edge_index_aug

# 边扰动
def augment_edge_perturb(features, edge_index, drop_prob=0.1):
    """
    边扰动：先以一定概率删除部分边，再随机添加一些新的边，
    添加的新边数量与被删除边的数量相当。
    """
    if isinstance(drop_prob, (list, tuple)):
        drop_prob = float(drop_prob[0])
    num_edges = edge_index.shape[1]
    num_nodes = features.shape[0]
    mask = (torch.rand(num_edges, device=edge_index.device) > drop_prob)
    edge_index_dropped = edge_index[:, mask]
    num_dropped = num_edges - mask.sum().item()
    if num_dropped > 0:
        new_edges = torch.randint(0, num_nodes, (2, num_dropped), device=features.device)
        edge_index_aug = torch.cat([edge_index_dropped, new_edges], dim=1)
    else:
        edge_index_aug = edge_index_dropped
    return features, edge_index_aug

def augment_data(data, aug_method="feature", aug_ratio=0.1):
    """
    根据指定的增强方式对图数据进行增强，返回增强后的节点特征和 edge_index。
    参数:
      aug_method: "feature"（特征扰动）, "node_drop", "edge_drop", "edge_perturb"
      aug_ratio: 控制增强强度（例如噪声水平或丢弃比例）
    """
    if aug_method == "feature":
        x_aug = perturb_features(data.x, noise_level=aug_ratio)
        edge_index_aug = data.edge_index  # 图结构不变
        return x_aug, edge_index_aug
    elif aug_method == "node_drop":
        return augment_node_drop(data.x, data.edge_index, drop_prob=aug_ratio)
    elif aug_method == "edge_drop":
        return augment_edge_drop(data.x, data.edge_index, drop_prob=aug_ratio)
    elif aug_method == "edge_perturb":
        return augment_edge_perturb(data.x, data.edge_index, drop_prob=aug_ratio)
    else:
        raise ValueError(f"Unknown augmentation method: {aug_method}")

#########################################
# 3.1 辅助函数：提取 mini-batch 子图
#########################################
def get_mini_batch_data(data, batch_node_idx, num_hops):
    """
    对单图中一小批节点（batch_node_idx）提取 k-hop 子图。
    使用 torch_geometric.utils.k_hop_subgraph 进行子图提取，并 relabel 节点。
    返回：
      sub_data: 包含子图的 Data 对象（x, edge_index, y 以及 mask 可选）
      mapping: 一个长整型张量，指示子图中哪一部分对应原始 batch_node_idx（目标节点在子图中的索引）
    """
    # k_hop_subgraph 返回：(subset, sub_edge_index, mapping, edge_mask)
    subset, sub_edge_index, mapping, _ = k_hop_subgraph(
        node_idx=batch_node_idx, num_hops=num_hops, edge_index=data.edge_index, relabel_nodes=True)
    sub_data = Data(x=data.x[subset], edge_index=sub_edge_index, y=data.y[subset])
    # 如果原始 data 定义了 train/val/test mask，则在子图中也保留（注意：mask 按 subset 索引提取）
    if hasattr(data, 'train_mask'):
        sub_data.train_mask = data.train_mask[subset]
    if hasattr(data, 'val_mask'):
        sub_data.val_mask = data.val_mask[subset]
    if hasattr(data, 'test_mask'):
        sub_data.test_mask = data.test_mask[subset]
    return sub_data, mapping

#########################################
# 4. 训练函数（预训练 + 微调）——mini-batch 版（基于 k_hop_subgraph）
#########################################
def pretrain_model(data, model, optimizer, criterion_contrast, num_epochs=200, aug_method="feature", aug_ratio=0.1, batch_size=64):
    """
    预训练阶段：仅使用对比损失训练模型（不计算交叉熵损失）。
    采用 mini-batch 方式，先对训练集中的一批目标节点提取 k-hop 子图，
    再对该子图进行数据增强生成两视图，最后计算对比损失（仅对 batch 中的目标节点计算）。
    使用 tqdm 显示每个 epoch 以及 batch 的进度和损失。
    """
    best_f1 = 0.0
    best_model_state = None
    train_idx = data.train_mask.nonzero(as_tuple=False).view(-1).tolist()
    num_hops = len(model.convs)  # 以模型层数作为子图的 hop 数

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        count = 0
        loader = DataLoader(train_idx, batch_size=batch_size, shuffle=True)
        # 使用 tqdm 包裹 batch 迭代
        pbar = tqdm(loader, desc=f"Pretrain Epoch {epoch+1}/{num_epochs}", leave=False)
        for batch in pbar:
            batch = batch.clone().detach().to(device)
            sub_data, mapping = get_mini_batch_data(data, batch, num_hops)
            sub_data = sub_data.to(device)
            optimizer.zero_grad()
            x_aug1, edge_index1 = augment_data(sub_data, aug_method, aug_ratio)
            x_aug2, edge_index2 = augment_data(sub_data, aug_method, aug_ratio)
            embedding_aug1 = model.encode(x_aug1, edge_index1)
            embedding_aug2 = model.encode(x_aug2, edge_index2)
            target_emb1 = embedding_aug1[mapping]
            target_emb2 = embedding_aug2[mapping]
            features_aug = torch.stack([target_emb1, target_emb2], dim=1)
            loss = criterion_contrast(features_aug)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * len(batch)
            count += len(batch)
            avg_loss = total_loss / count
            pbar.set_postfix(batch_loss=f"{loss.item():.4f}", avg_loss=f"{avg_loss:.4f}")
        print(f"Pretrain Epoch {epoch+1}/{num_epochs}, Contrast Loss: {avg_loss:.4f}")

        # 验证阶段
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out[data.val_mask].argmax(dim=1)
            true = data.y[data.val_mask]
            val_f1 = f1_score(true.cpu(), preds.cpu(), average="macro")
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    return best_model_state

def fine_tune_model(data, model, optimizer, criterion_ce, num_epochs=50, batch_size=64):
    """
    微调阶段：仅使用交叉熵损失进行训练（不计算对比损失）。
    同样采用 mini-batch（基于 k_hop 子图）方式训练，损失仅在目标节点上计算。
    使用 tqdm 实时显示每个 epoch 与 batch 的训练进度和损失信息。
    """
    best_f1 = 0.0
    best_model_state = None
    train_idx = data.train_mask.nonzero(as_tuple=False).view(-1).tolist()
    num_hops = len(model.convs)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        count = 0
        loader = DataLoader(train_idx, batch_size=batch_size, shuffle=True)
        pbar = tqdm(loader, desc=f"Fine-tune Epoch {epoch+1}/{num_epochs}", leave=False)
        for batch in pbar:
            batch = batch.clone().detach().to(device)
            sub_data, mapping = get_mini_batch_data(data, batch, num_hops)
            sub_data = sub_data.to(device)
            optimizer.zero_grad()
            out = model(sub_data)
            loss = criterion_ce(out[mapping], sub_data.y[mapping])
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item() * len(batch)
            count += len(batch)
            avg_loss = total_loss / count
            pbar.set_postfix(batch_loss=f"{loss.item():.4f}", avg_loss=f"{avg_loss:.4f}")
        print(f"Fine-tune Epoch {epoch+1}/{num_epochs}, CE Loss: {avg_loss:.4f}")

        # 验证阶段
        model.eval()
        with torch.no_grad():
            out = model(data)
            preds = out[data.val_mask].argmax(dim=1)
            true = data.y[data.val_mask]
            val_f1 = f1_score(true.cpu(), preds.cpu(), average="macro")
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model_state = model.state_dict()

    return best_model_state

def two_stage_train_model(data, model, optimizer, optimizer_ft, criterion_ce, criterion_contrast,
                          pretrain_epochs, finetune_epochs, aug_method="feature", aug_ratio=0.1,
                          batch_size=64):
    """
    两阶段训练：
      第一阶段：预训练（仅用对比损失，mini-batch 方式）；
      第二阶段：微调（仅用交叉熵损失，mini-batch 方式）。
    """
    print("========== 开始预训练阶段 ==========")
    best_pretrain_state = pretrain_model(data, model, optimizer, criterion_contrast,
                                         num_epochs=pretrain_epochs,
                                         aug_method=aug_method, aug_ratio=aug_ratio,
                                         batch_size=batch_size)
    model.load_state_dict(best_pretrain_state)

    print("========== 开始微调阶段 ==========")
    best_finetune_state = fine_tune_model(data, model, optimizer_ft,
                                          criterion_ce, num_epochs=finetune_epochs,
                                          batch_size=batch_size)
    return best_finetune_state

#########################################
# 5. 封装随机采样超参数组合的函数
#########################################
def get_continuous_candidates(start, stop, step, decimals):
    """
    生成从 start 到 stop（含）之间，以 step 为步长的候选列表，并保留指定小数位数。
    """
    num_steps = int((stop - start) / step) + 1
    return [round(start + i * step, decimals) for i in range(num_steps)]

def get_random_hyperparameter_combinations(n_iter):
    """
    随机生成 n_iter 个超参数组合，每个组合包含：
      (threshold, random_state, num_layers, hidden_channels, finetune_lr, pretrain_lr,
       gamma, alpha_value, aug_method, aug_ratio, pretrain_epochs, temperature,
       model_type, dropout_rate)
    """
    # 离散变量候选列表
    discrete_candidates = {
        'random_state': list(range(0, 40, 10)),
        'num_layers': [2, 3, 4],
        'hidden_channels': list(range(20, 300, 20)),
        'finetune_lr': [0.001],
        'pretrain_lr': [0.0001, 0.00001, 0.000001],
        'aug_method': ["feature", "node_drop", "edge_drop", "edge_perturb"],
        'pretrain_epochs': [5],
        'model_type': ["GraphSAGE", "GAT"],
        'dropout_rate': [0.1, 0.2, 0.3]
    }

    # 连续变量候选区间及步长（注意：部分变量保留 2 位小数，其余保留 1 位小数）
    continuous_candidates = {
        'threshold': get_continuous_candidates(0.05, 0.2, 0.01, 2),
        'gamma': get_continuous_candidates(1, 4, 0.1, 1),
        # 'alpha_value': get_continuous_candidates(0.1, 0.9, 0.1, 1),
        'alpha_value': [0.22, 0.78],
        'aug_ratio': get_continuous_candidates(0.05, 0.25, 0.01, 2),
        'temperature': get_continuous_candidates(0.05, 0.1, 0.01, 2)
    }

    combinations = []
    for _ in range(n_iter):
        # 随机采样连续变量
        threshold   = random.choice(continuous_candidates['threshold'])
        gamma       = random.choice(continuous_candidates['gamma'])
        alpha_value = random.choice(continuous_candidates['alpha_value'])
        aug_ratio   = random.choice(continuous_candidates['aug_ratio'])
        temperature = random.choice(continuous_candidates['temperature'])
        
        # 随机采样离散变量
        random_state    = random.choice(discrete_candidates['random_state'])
        num_layers      = random.choice(discrete_candidates['num_layers'])
        hidden_channels = random.choice(discrete_candidates['hidden_channels'])
        finetune_lr     = random.choice(discrete_candidates['finetune_lr'])
        pretrain_lr     = random.choice(discrete_candidates['pretrain_lr'])
        aug_method      = random.choice(discrete_candidates['aug_method'])
        pretrain_epochs = random.choice(discrete_candidates['pretrain_epochs'])
        model_type      = random.choice(discrete_candidates['model_type'])
        dropout_rate    = random.choice(discrete_candidates['dropout_rate'])
        
        # 构造超参数组合（顺序与注释中保持一致）
        combination = (
            threshold,      # 阈值
            random_state,   # 随机种子
            num_layers,     # 图卷积层数
            hidden_channels,# 第一层隐藏单元数
            finetune_lr,    # 微调学习率
            pretrain_lr,    # 预训练学习率
            gamma,          # gamma 参数
            alpha_value,    # alpha 参数
            aug_method,     # 增强方式
            aug_ratio,      # 增强比例
            pretrain_epochs,# 预训练轮数
            temperature,    # 对比学习温度
            model_type,     # 模型类型
            dropout_rate    # dropout 概率
        )
        combinations.append(combination)
        
    return combinations

#########################################
# 6. 随机搜索超参数并评估模型
#########################################
def grid_search(X, y, train_mask, valid_mask, test_mask, n_iter):
    best_acc = 0.0
    best_overall_model_state = None
    best_overall_params = None

    print("Start random search with {} combinations...".format(n_iter))
    hyperparam_combos = get_random_hyperparameter_combinations(n_iter)
    for i, (threshold, random_state, num_layers, hidden_channels, finetune_lr,
            pretrain_lr, gamma, alpha_value, aug_method, aug_ratio, pretrain_epochs, temperature,
            model_type, dropout_rate) in enumerate(hyperparam_combos):
        print(f"\nTesting combination {i+1}: threshold={threshold:.4f}, random_state={random_state}, "
              f"layers={num_layers}, hidden_channels={hidden_channels}, finetune_lr={finetune_lr}, "
              f"pretrain_lr={pretrain_lr}, gamma={gamma:.4f}, alpha={alpha_value:.4f}, aug_method={aug_method}, "
              f"aug_ratio={aug_ratio:.4f}, pretrain_epochs={pretrain_epochs}, temperature={temperature:.4f}, "
              f"model_type={model_type}, dropout_rate={dropout_rate:.4f}")

        # 假设函数 adjacency_to_edge_index 已实现
        edge_index = adjacency_to_edge_index(adjacency_matrix, threshold).to(device)
        X_tensor = torch.tensor(X.values, dtype=torch.float)
        y_tensor = torch.tensor(y.values, dtype=torch.long)
        data = Data(x=X_tensor, y=y_tensor, edge_index=edge_index,
                    train_mask=train_mask, val_mask=valid_mask, test_mask=test_mask).to(device)

        # 根据 model_type 选择模型
        if model_type == "GraphSAGE":
            model = GraphSAGE(in_channels=X.shape[1], hidden_channels=hidden_channels,
                              out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
        elif model_type == "GAT":
            model = GAT(in_channels=X.shape[1], hidden_channels=hidden_channels,
                        out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
        else:
            raise ValueError(f"Unknown model type: {model_type}")
        
        optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_lr, weight_decay=5e-4)
        optimizer_ft = torch.optim.Adam(model.parameters(), lr=finetune_lr, weight_decay=5e-4)

        # 使用普通的交叉熵损失替代 FocalLoss
        criterion_ce = torch.nn.CrossEntropyLoss()
        criterion_contrast = SupConLoss(temperature=temperature)

        best_model_epoch = two_stage_train_model(data, model, optimizer, optimizer_ft, criterion_ce,
                                                 criterion_contrast, pretrain_epochs=pretrain_epochs, finetune_epochs=10,
                                                 aug_method=aug_method, aug_ratio=aug_ratio, batch_size=128)
        model.load_state_dict(best_model_epoch)
        model.eval()
        with torch.no_grad():
            test_out = model(data)
            preds = test_out[data.test_mask].argmax(dim=1)
            test_acc = accuracy_score(data.y[data.test_mask].cpu(), preds.cpu())
        print(f"Test Accuracy for current combination: {test_acc:.4f}")
        if test_acc > best_acc:
            best_acc = test_acc
            best_overall_model_state = best_model_epoch
            best_overall_params = (threshold, random_state, num_layers, hidden_channels,
                                   finetune_lr, pretrain_lr, gamma, alpha_value, aug_method, aug_ratio,
                                   pretrain_epochs, temperature, model_type, dropout_rate)

    return best_overall_params, best_overall_model_state

#########################################
# 7. 主程序：加载数据、随机搜索超参数、加载最佳模型并评估
#########################################
# 注意：此处假定 X, y, adjacency_matrix, train_mask, valid_mask, test_mask 已经提前加载好，
# 且 edge_index 为形状 [2, num_edges] 的 torch.tensor 对象。

best_params, best_model_state = grid_search(X, y, train_mask, valid_mask, test_mask, n_iter=100)
print("\nBest Hyperparameters:", best_params)

# 解包最佳超参数
(threshold, random_state, num_layers, hidden_channels,
 finetune_lr, pretrain_lr, gamma, alpha_value, aug_method, aug_ratio,
 pretrain_epochs, temperature, model_type, dropout_rate) = best_params

X_tensor = torch.tensor(X.values, dtype=torch.float).to(device)
y_tensor = torch.tensor(y.values, dtype=torch.long).to(device)
edge_index = adjacency_to_edge_index(adjacency_matrix, threshold).to(device)
data = Data(x=X_tensor, y=y_tensor, edge_index=edge_index, train_mask=train_mask.to(device), val_mask=valid_mask.to(device), test_mask=test_mask.to(device))

# 根据最终选出的模型类型构造模型
if model_type == "GraphSAGE":
    model = GraphSAGE(in_channels=X.shape[1], hidden_channels=hidden_channels,
                      out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
elif model_type == "GAT":
    model = GAT(in_channels=X.shape[1], hidden_channels=hidden_channels,
                out_channels=len(np.unique(y)), num_layers=num_layers, dropout_rate=dropout_rate).to(device)
else:
    raise ValueError(f"Unknown model type: {model_type}")

model.load_state_dict(best_model_state)
model.eval()

with torch.no_grad():
    test_out = model(data)
    preds = test_out[data.test_mask].argmax(dim=1)
    true_labels = data.y[data.test_mask]
    report = classification_report(true_labels.cpu(), preds.cpu(), target_names=["Class 0", "Class 1"], digits=4)
    test_precision = precision_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_recall = recall_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_f1 = f1_score(true_labels.cpu(), preds.cpu(), average="macro")
    test_acc = accuracy_score(true_labels.cpu(), preds.cpu())
    
    print("\nBest Model Classification Report on Test Set:")
    print(report)
    print("Best Model Test Set Metrics:")
    print(f"Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}, Accuracy: {test_acc:.4f}")


Start random search with 100 combinations...

Testing combination 1: threshold=0.1000, random_state=20, layers=3, hidden_channels=80, finetune_lr=0.001, pretrain_lr=0.0001, gamma=1.1000, alpha=0.7800, aug_method=edge_perturb, aug_ratio=0.1400, pretrain_epochs=5, temperature=0.0600, model_type=GAT, dropout_rate=0.2000
邻接矩阵转换完成！
Edge index 维度: torch.Size([2, 2058380])
========== 开始预训练阶段 ==========


Pretrain Epoch 1/5, Contrast Loss: 6.9863


Pretrain Epoch 2/5, Contrast Loss: 6.6198


Pretrain Epoch 3/5, Contrast Loss: 6.4599


Pretrain Epoch 4/5, Contrast Loss: 6.4087


...

Fine-tune Epoch 7/10, CE Loss: 0.7534


Fine-tune Epoch 8/10, CE Loss: 0.6226


Fine-tune Epoch 9/10, CE Loss: 0.6292


Fine-tune Epoch 10/10, CE Loss: 0.5844
Test Accuracy for current combination: 0.7788

Testing combination 12: threshold=0.0700, random_state=30, layers=3, hidden_channels=240, finetune_lr=0.001, pretrain_lr=1e-06, gamma=2.4000, alpha=0.7800, aug_method=node_drop, aug_ratio=0.0500, pretrain_epochs=5, temperature=0.1000, model_type=GAT, dropout_rate=0.3000
邻接矩阵转换完成！
Edge index 维度: torch.Size([2, 4710448])
========== 开始预训练阶段 ==========


RuntimeError: CUDA out of memory. Tried to allocate 2.87 GiB (GPU 0; 23.70 GiB total capacity; 16.92 GiB already allocated; 2.55 GiB free; 19.45 GiB reserved in total by PyTorch)